In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
postPorID = {}
with open('subs/userPorPostId.csv') as f:
    for line in f:
        lista = line.split(",")
        postId = lista[2][:-1]
        user = lista[1]
        postPorID[postId] = user

In [11]:
SUB = "the_donald"
FILE = "subs/RC_2017-04."+SUB+".comments.csv"
OUTPUT_FILE = "grafos/"+SUB+".gml"
comments =pd.read_csv(FILE,index_col=0)

In [12]:
print(comments.count())
comments = comments[comments["user"] != "[deleted]"]
comments.set_index('id', inplace=True)

user      200000
id        200000
parent    200000
dtype: int64


In [13]:
postPorID

{'postId': 'user',
 '62p82p': 'nobreath',
 '62p82q': 'mplewis1',
 '62p82r': 'cooltrain7',
 '62p82s': 'littlesmol',
 '62p82t': 'RedneckWilly',
 '62p82u': '[deleted]',
 '62p82v': 'xKONIGSTIGERx',
 '62p82w': 'SativaSammy',
 '62p82x': '[deleted]',
 '62p82y': 'OctoJoey',
 '62p82z': 'Barefoot_Raphsode',
 '62p830': 'Luzargx',
 '62p831': 'Exemalti',
 '62p832': '[deleted]',
 '62p833': 'girt-by-sea',
 '62p834': '[deleted]',
 '62p835': 'pushshift_bot',
 '62p836': 'droppadabass',
 '62p837': 'Mobius1337',
 '62p838': 'hperce',
 '62p839': 'ranalog',
 '62p83a': 'alle15minuten',
 '62p83b': '[deleted]',
 '62p83c': 'Azure_Eagle',
 '62p83d': 'Colonel-Turtle',
 '62p83e': 'Yarkika',
 '62p83f': 'candyrockstar',
 '62p83g': 'ImagesOfNetwork',
 '62p83h': 'uasmr_bot',
 '62p83i': 'qtheconquerer',
 '62p83k': '[deleted]',
 '62p83l': 'Blasterly',
 '62p83m': 'super_aardvark',
 '62p83n': 'kirig19',
 '62p83o': 'The_Salty_Engineer',
 '62p83p': '[deleted]',
 '62p83q': '[deleted]',
 '62p83r': '[deleted]',
 '62p83s': '[del

In [14]:
commentsNulos = "__NOT__"
def getParent(commentId):
    tipo, user = commentId.split("_")
    try:
        if tipo == "t1":
            res = comments.loc[user]["user"]
        else:
            res = postPorID[user]
    except KeyError:
        res = commentsNulos
    return res

In [15]:
userParents = [getParent(x) for x in comments["parent"].tolist()]
comments["padre"] = userParents

In [16]:
comments = comments[comments["padre"] != commentsNulos]
comments = comments[comments["user"] != "[deleted]"]
comments = comments[comments["padre"] != "[deleted]"]

In [17]:
users = comments["user"].unique()
print(len(users),
len(comments))

21418 157012


In [32]:
M=nx.from_pandas_edgelist(comments, "user", 'padre', create_using=nx.MultiDiGraph())
g=nx.MultiDiGraph()
for u,v in M.edges():
    if g.has_edge(u,v):
        g[u][v][0]['weight'] += 1
    else:
        g.add_edge(u, v, weight=1)
g.remove_edges_from([x for x in g.selfloop_edges()])
g.add_nodes_from(users)
nx.write_gml(g, OUTPUT_FILE)

In [134]:
len(g.nodes())

3809

In [35]:
fd=nx.Graph()
for u,v in g.edges():
    if g.has_edge(u,v) and g.has_edge(v,u):
        fd.add_edge(u, v, color='red')
    else:
        fd.add_edge(u, v, color='blue')
nx.write_gml(g, OUTPUT_FILE+"fd.gml")